# Domácí úkol:
Pomocí vlastních tříd a dědičnosti vytvořte "soubor tříd" = nástroj pro derivaci funkcí.

## Jak to má fungovat:

- derivace je jednoduše algoritmizovatelná, neboť se řídí prakticky pouze třemi pravidly (podílu se dá vyhnout pomocí násobení funkcí $\frac{1}{f(x)}$):
    - derivace součtu je součet derivací
    - pravidlo pro derivaci součinu
    - derivace funkce je derivovaná funkce násobená derivovaným argumentem

- tedy stačí implementovat tyto tři základní pravidla a pro každou elementární funkci (sin, cos, exp, ln, x^n, ...) vědět jak vypadá její derivace
- celý tento proces je možné reprezentovat pomocí tříd a dědičnosti
- trik je v tom, že se výrazy automaticky skládají do stromu (pro nás v podstatě skrytého), a tedy každý výraz je na venek pouze buď sčítání, násobení, nebo elementární funkce (a o jejich argumentech platí totéž)

## Podrobnější zadání:
Konkrétněji:
- Vytvořte třídu **Operator**
    - Třída bude mít metody:
        - `__init__()` - inicializuje operátor (zde stačí `pass`)
        - `__str__()` - vrací řetězec reprezentující operátor
        - `__repr__()` - vrací řetězec reprezentující operátor
- Vytvořte třídu **BinaryOperator**, která bude dědit od třídy Operator
    - Třída bude mít atributy:
        - `levy_argument` - levý operand
        - `pravy_argument` - pravý operand
    - Třída bude mít metody:
        - `__init__()` - inicializuje operátor, přiřadí argumenty
        - `__str__()` - vrací řetězec reprezentující operátor
        - `__repr__()` - vrací řetězec reprezentující operátor
        - `diff()` - vrací derivaci operátoru
- Vytvořte třídu **UnaryOperator**, která bude dědit od třídy Operator
    - Třída bude mít atributy:
        - `argument` - argument funkce 
    - Třída bude mít metody:
        - `__init__()` - inicializuje operátor, přiřadí argument
        - `__str__()` - vrací řetězec reprezentující operátor
        - `__repr__()` - vrací řetězec reprezentující operátor
        - `diff()` - vrací derivaci operátoru

Pro počítání derivací použijte standardní pravidla:
- derivace součtu je součet derivací
- pravidlo pro derivaci součinu
- derivace funkce je derivovaná funkce násobená derivovaným argumentem

Pro **BinaryOperator** a **UnaryOperator** implementujte také metody implementující standardní operátory:
- `__add__()` - implementuje operátor `+`
- `__sub__()` - implementuje operátor `-`
- `__mul__()` - implementuje operátor `*`

Pro **BinaryOperator** implementujte dvě dceřiné třídy:
- **Addition** - implementuje operátor `+`
- **Multiplication** - implementuje operátor `*`

Pro **UnaryOperator** implementujte tyto dceřiné třídy:
- **exp** - implementuje funkci `exp`
- **sin** - implementuje funkci `sin`
- **cos** - implementuje funkci `cos`
- **ln** - implementuje funkci `ln`
- **frc** - implementuje funkci `1/x`
- **identity** - implementuje identitu `x`
- **cst** - implementuje konstantu `c`
- **mocnina** - implementuje mocninu `x^n`

U všech funckí implementujte adekvátně metodu `diff()` a `__str__()`, tak aby se funkce vypisovaly v přehledné podobě.

Ozkoušejte na následujících funkcích:
- $sin(x + cos(x*x)) + 1$
- $x^2 + 2x + 1 + 2x^2$
- $x^2 + 2x + exp(x)$

In [17]:
# kostra k doplnění, místa označená TODO doplňte

class Operator:
    def __init__(self):
        pass
    
    def __str__(self):
        return type(self).__name__
    
    def __repr__(self):
        return self.__str__()
    
    def diff(self):
        pass
    
    def __add__(self, other):
        # TODO (použijte Addition)
        return Addition(self, other)
    
    def __sub__(self, other):
        # TODO (použijte Addition a Multiplication se zápornou konstantu)
        return Addition(self, Multiplication(other, cst(-1)))
    
    def __mul__(self, other):
        # TODO (použijte Multiplication)
        return Multiplication(self, other)
    
class BinaryOperator(Operator):
    def __init__(self, levy_argument, pravy_argument):
        self.levy_argument = levy_argument
        self.pravy_argument = pravy_argument

class UnaryOperator(Operator):
    def __init__(self, argument):
        self.argument = argument
    
    def __str__(self):
        return type(self).__name__ + "(" + str(self.argument) + ")"
    
    def simplify(self):
        self.argument = self.argument.simplify()
        return self
        
class Addition(BinaryOperator):
    def __str__(self):
        # TODO vypíšeme něco jako (levy_argument)+(pravy_argument)
        return f"({self.levy_argument} + {self.pravy_argument})"
    def diff(self):
        # TODO
        return Addition(self.levy_argument.diff(), self.pravy_argument.diff())
    def simplify(self):
        self.levy_argument = self.levy_argument.simplify()
        self.pravy_argument = self.pravy_argument.simplify()
        
        if isinstance(self.levy_argument, cst) and self.levy_argument.value == 0:
            return self.pravy_argument
        elif isinstance(self.pravy_argument, cst) and self.pravy_argument.value == 0:
            return self.levy_argument
        elif isinstance(self.levy_argument, cst) and isinstance(self.pravy_argument, cst):
            return cst(self.levy_argument.value + self.pravy_argument.value)
        elif self.levy_argument == self.pravy_argument:
            return Multiplication(2, self.levy_argument)
        else:
            return self
    
class Multiplication(BinaryOperator):
    def __str__(self):
        # TODO vypíšeme něco jako (levy_argument)*(pravy_argument)
        return f"{self.levy_argument} * {self.pravy_argument}"
    def diff(self):
        # TODO
        return Addition(
            Multiplication(self.levy_argument.diff(), self.pravy_argument),
            Multiplication(self.levy_argument, self.pravy_argument.diff())
        )
    def simplify(self):
        self.levy_argument = self.levy_argument.simplify()
        self.pravy_argument = self.pravy_argument.simplify()

        if isinstance(self.levy_argument, cst) and self.levy_argument.value == 0:
            return cst(0)
        elif isinstance(self.pravy_argument, cst) and self.pravy_argument.value == 0:
            return cst(0)
        elif isinstance(self.levy_argument, cst) and isinstance(self.pravy_argument, cst):
            return cst(self.levy_argument.value * self.pravy_argument.value)
        elif isinstance(self.levy_argument, cst) and self.levy_argument.value == 1:
            return self.pravy_argument.simplify()
        elif isinstance(self.pravy_argument, cst) and self.pravy_argument.value == 1:
            return self.levy_argument.simplify()
        elif self.levy_argument == self.pravy_argument:
            return mocnina(self.levy_argument.simplify(), 2)
        else:
            return self
    
class cst(UnaryOperator):
    def __init__(self, value):
        self.value = value
    def __str__(self):
        # TODO vypíšeme něco jako value
        return str(self.value)
    def diff(self):
        # TODO
        return cst(0)
    def simplify(self):
        return self
    
class identity(UnaryOperator):
    def __init__(self):
        pass
    def __str__(self):
        # TODO vypíšeme něco jako x   
        return "x"
    def diff(self):
        # TODO
        return cst(1)
    def simplify(self):
        return self
    
class mocnina(UnaryOperator):
    def __init__(self, argument, exponent):
        self.argument = argument
        self.exponent = exponent
    def __str__(self):
        # TODO vypíšeme něco jako argument^exponent
        return f"{self.argument}^{self.exponent}"
    def diff(self):
        # TODO
        return Multiplication(
            cst(self.exponent),
            mocnina(self.argument, self.exponent - 1)
        )
    def simplify(self):
        if self.exponent == 0:
            return cst(1)
        elif self.exponent == 1:
            return self.argument.simplify()
        else:
            return self
    
class sin(UnaryOperator):
    def diff(self):
        # TODO
        return Multiplication(self.argument.diff(), cos(self.argument))

class cos(UnaryOperator):
    def diff(self):
        # TODO
        return Multiplication(
            self.argument.diff(),
            Multiplication(cst(-1), sin(self.argument))
        )

class exp(UnaryOperator):
    def diff(self):
        # TODO
        return Multiplication(self.argument.diff(), exp(self.argument))
    
class ln(UnaryOperator):
    def diff(self):
        # TODO
        return Multiplication(self.argument.diff(), frc(self.argument))

class frc(UnaryOperator):
    def __str__(self):
        # TODO vypíšeme něco jako 1/(argument)
        return f"1/({self.argument})"
    
    def diff(self):
        # TODO zde je to asi vzhledem k tomu jak jsme si systém navrhli trochu složitější
        # bude to zvlášť "-1 *" zvlášť zlomek s argumentem ve tvaru mocniny krát derivace argumentu
        return Multiplication(
            self.argument.diff(),
            Multiplication(
                cst(-1),
                frc(mocnina(self.argument, cst(2)))
            )
        )

In [28]:
# takto by se to mělo používat
x = identity()
# sin(x + cos(x*x)) + 1
f1 = sin(x + cos(x*x)) + cst(1)
print(f"Original:\t\t {f1}")
print(f"Original diff:\t\t {f1.diff()}")
print(f"Simplified over og:\t {f1.simplify()}")
print(f"Simplified over diff:\t {f1.diff().simplify()}")

Original:		 (sin((x + cos(x * x))) + 1)
Original diff:		 ((1 + (1 * x + x * 1) * -1 * sin(x * x)) * cos((x + cos(x * x))) + 0)
Simplified over og:	 (sin((x + cos(x^2))) + 1)
Simplified over diff:	 (1 + 2 * x * -1 * sin(x^2)) * cos((x + cos(x^2)))


můj výstup byl:

`sin(x + cos((x)*(x))) + 1`

`(cos(x + cos((x)*(x))))*(1 + ((-1)*(sin((x)*(x))))*((1)*(x) + (x)*(1))) + 0`

In [26]:
x = identity()
# x ^ 2 + 2x + 1 + 2x ^ 2
f2 = mocnina(x,2) + cst(2)*x + cst(1) + cst(2)*mocnina(x,2)
print(f"Original:\t\t {f2}")
print(f"Original diff:\t\t {f2.diff()}")
print(f"Simplified over og:\t {f2.simplify()}")
print(f"Simplified over diff:\t {f2.diff().simplify()}")

Original:		 (((x^2 + 2 * x) + 1) + 2 * x^2)
Original diff:		 (((2 * x^1 + (0 * x + 2 * 1)) + 0) + (0 * x^2 + 2 * 2 * x^1))
Simplified over og:	 (((x^2 + 2 * x) + 1) + 2 * x^2)
Simplified over diff:	 ((2 * x + 2) + 2 * 2 * x)


můj výstup byl:

`(x)^2 + (2)*(x) + 1 + (2)*((x)^2)`

`((2)*((x)^1))*(1) + (0)*(x) + (2)*(1) + 0 + (0)*((x)^2) + (2)*(((2)*((x)^1))*(1))`

In [27]:
x = identity()
# x ^ 2 + 2x + exp(x)
f3 = mocnina(x,2) + cst(2)*x + exp(x)
print(f"Original:\t\t {f3}")
print(f"Original diff:\t\t {f3.diff()}")
print(f"Simplified over og:\t {f3.simplify()}")
print(f"Simplified over diff:\t {f3.diff().simplify()}")


Original:		 ((x^2 + 2 * x) + exp(x))
Original diff:		 ((2 * x^1 + (0 * x + 2 * 1)) + 1 * exp(x))
Simplified over og:	 ((x^2 + 2 * x) + exp(x))
Simplified over diff:	 ((2 * x + 2) + exp(x))


můj výstup byl:

`(x)^2 + (2)*(x) + exp(x)`

`((2)*((x)^1))*(1) + (0)*(x) + (2)*(1) + (exp(x))*(1)`


# Bonusový úkol:
Doplňte ke všem třídám metodu `simplify()`, která provede zjednodušení výrazu. Například:
- součet s nulou vrátí druhý operand
- součin s nulou vrátí nulu
- mocnina s exponentem 0 vrátí 1
- mocnina s exponentem 1 vrátí první argument
- součín dvou stejných argumentů vrátí mocninu s exponentem 2
- součin konstant vrátí konstantu se součinem
- ...